In [3]:
import copy

with open('data/day17.csv') as f:
    input = [1 if i == '>' else -1 for i in f.read().strip()]

rocks =    {0: [[3,0], [4,0], [5,0], [6,0]],            # MINUS
            1: [[4,0], [3,1], [4,1], [5,1], [4,2]],     # PLUS
            2: [[3,0], [4,0], [5,0], [5,1], [5,2]],     # REVERSED L
            3: [[3,0], [3,1], [3,2], [3,3]],            # PIPE
            4: [[3,0], [4,0], [3,1], [4,1]]}            # SQUARE

rockpos =  {0: 0,   # MINUS
            1: 2,   # PLUS
            2: 2,   # REVERSED L
            3: 3,   # PIPE
            4: 1}   # SQUARE

inputpos = 0
amountofinputs = len(input)
solid = {(1,0), (2,0), (3,0), (4,0), (5,0), (6,0), (7,0)}
heigth = 0

def push_rock(coords):
    global inputpos
    blowdirection = input[inputpos % amountofinputs]
    stuckhorizontal = False
    for coord in coords:
        if (coord[0] + blowdirection, coord[1]) in solid or blowdirection + coord[0] == 8 or blowdirection + coord[0] == 0:
            stuckhorizontal = True
            break
    if not stuckhorizontal:
        for coord in coords:
            coord[0] += blowdirection
    inputpos += 1
    return coords

def fall_rock(coords, localheigth):
    global solid, heigth
    stuck = False
    for coord in coords:
        if (coord[0], coord[1] - 1) in solid:
            stuck = True
            heigth = max(localheigth, heigth)
            break
    if stuck == False:
        for coord in coords:
            coord[1] -= 1
        localheigth -= 1
    return coords, stuck, localheigth

def new_rock(i):
    global heigth
    localheigth = rockpos[i % len(rockpos)]
    coords = copy.deepcopy(rocks[i % len(rocks)])
    for coord in coords:
        coord[1] += heigth + 4
    localheigth += heigth + 4
    return coords, localheigth

def clean_solid(solid, heigth, newheigth):
    minheigth = heigth
    for j in range(1,8):
        minheigth = min(minheigth, max([i[1] for i in solid if i[0] == j]))
    solid = set([(i[0], i[1] - minheigth) for i in solid if i[1] >= minheigth])
    newheigth += minheigth
    heigth -= minheigth
    return solid, heigth, newheigth

newheigth = 0
status = []
status2 = []
i = 0
patternlength = 1
patternheigth = 1
patternfound = 0
numberofrocks = 1000000000000

while i < numberofrocks:
    coords, localheigth = new_rock(i)
    stuck = False

    if i % 5 == 0 and i != 0 and patternfound <= 2:
        if [heigth, inputpos % amountofinputs] in status2:
            status = copy.deepcopy(status2)
            status2 = []
        status2.append([heigth, inputpos % amountofinputs])
        if status == status2 and patternfound == 0:
            patternfound = 1
            patternlength = i
            patternheigth = heigth+newheigth
        elif status == status2 and patternfound == 1:
            patternfound = 2
            patternlength = i - patternlength
            patternheigth = heigth + newheigth - patternheigth
    if patternfound == 2 and (numberofrocks - i - 1) // patternlength > 0:
            newheigth += patternheigth * ((numberofrocks - i - 1) // patternlength)
            i += patternlength * ((numberofrocks - i - 1) // patternlength)
    else:
        while stuck == False:
            coords = push_rock(coords)
            coords, stuck, localheigth = fall_rock(coords, localheigth)
        for coord in coords:
            solid.add((coord[0], coord[1]))
        solid, heigth, newheigth = clean_solid(solid, heigth, newheigth)
        i += 1

print(newheigth + heigth)

1575811209487
